# tests format - pointer

In [11]:
from json_ntv import Ntv
from jsonpointer import resolve_pointer

example = {'data': {'a': 1, 'test': 'ok'}, 'pointer': '/test', 'canonical': '/1'}
example = {'data': [{'a': 1}, 'ok'],       'pointer': '/1',    'canonical': '/1'}
example = {'data': {'a': 1, '1': 'ok'},    'pointer': '/1',    'canonical': '/1'}
example = {'data': ['a', 'ok'],            'pointer': '/1',    'canonical': '/1'}

data      = example['data']
pointer   = example['pointer']
canonical = example['canonical']

ntv = Ntv.obj(data)
print(ntv[canonical] == ntv[pointer])

ntv[pointer].val == resolve_pointer(data, pointer) == 'ok'


True


True